# Automated Assessment Feedback Tool

**This project uses the meta-llama/Llama-3.1-8B-Instruct model** to generate structured feedback for question-and-answer pairs extracted from a Word document (Q1.docx). The feedback is then added as comments to the same document. It is designed to provide automated, structured feedback on responses.

##**Features**

*   **Extracts Q&A:** Reads question-answer pairs from a Word document.
*   **Generates Feedback:** Uses LLaMA 3.1 via Hugging Face's InferenceClient to produce structured feedback (positive points, praise, and suggestions for improvement).
*   **Adds Comments:** Inserts feedback as comments in the original Word document.
*   **Saves Output:** Updates the input document with feedback comments.

##**Prerequisites**

*   Python 3.8+
*   A Hugging Face account with an API token for accessing the LLaMA model.
*   A Word document (Q1.docx) with Q&A pairs formatted as Q1., Q2., etc., for questions and A1., A2., etc., for answers.

##**Usage**
*   Prepare the Input Document:
    *    Create a Word document (Q1.docx) with Q&A pairs. Example format:

    Q1. What is the safeguarding policy?

    A1. The policy ensures safety and compliance.
    
    Q2. How is safeguarding implemented?

    A2. Through training and regular audits.

##**Run the Notebook**
*   Open Comment_Project.ipynb in a Jupyter environment (e.g., Google Colab, JupyterLab).
*   Execute all cells to:
   *   Install dependencies.
   *   Extract Q&A pairs from Q1.docx.
   *   Generate feedback using LLaMA 3.1.
   *   Add feedback as comments to Q1.docx.

## **Output**
*   The modified Q1.docx will contain feedback as comments for each question.
*   Example feedback format:
   *   ✅ Positive points: Clear explanation of the policy.
   *   👏 Praise: Well-structured response.
   *   ⚠️ Suggestions for improvement: Include specific examples to strengthen the answer.



---



In [ ]:
!pip install -q transformers accelerate peft bitsandbytes torch python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Q1.docx to Q1.docx


# Generate AI FeedbacK Using LLM meta-llama/Llama-3.1

In [ ]:
import os
from docx import Document
from huggingface_hub import InferenceClient

# STEP 1: Set your Hugging Face token directly
os.environ["HF_TOKEN"] = "KEY"
# STEP 2: Initialize InferenceClient for LLaMA 3.1
client = InferenceClient(
    model="meta-llama/Llama-3.1-8B-Instruct",
    token=os.environ["HF_TOKEN"]
)

# STEP 3: Extract Q&A from Word
def extract_qa(filepath):
    doc = Document(filepath)
    pairs = []
    q, a = None, ""
    for para in doc.paragraphs:
        txt = para.text.strip()
        if txt.startswith("Q"):
            if q:
                pairs.append((q, a.strip()))
            q, a = txt, ""
        elif txt.startswith("A"):
            a += txt[1:].strip()
        else:
            a += "\n" + txt
    if q:
        pairs.append((q, a.strip()))
    return pairs

# STEP 4: Generate feedback using hosted LLaMA
def generate_feedback_llama(q, a):
    prompt = f"""
You are a safeguarding training assessor. Provide structured feedback for the answer below:

Question: {q}
Answer: {a}

Format:
✅ Positive points
👏 Praise (if any)
⚠️ Suggestions for improvement (if any)
"""
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=256,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# STEP 5: Write feedback to Word file
def write_feedback(feedbacks, out_path="Generated_Feedback.docx"):
    doc = Document()
    for i, f in enumerate(feedbacks, 1):
        doc.add_heading(f"Feedback {i}", level=2)
        doc.add_paragraph(f)
    doc.save(out_path)
    print(f"✅ Saved to {out_path}")

# STEP 6: Run the full pipeline
qa_pairs = extract_qa("Q1.docx")  # Adjust if filename changed
feedbacks = [generate_feedback_llama(q, a) for q, a in qa_pairs]
write_feedback(feedbacks)


✅ Saved to Generated_Feedback.docx


In [ ]:
files.download("Generated_Feedback.docx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# integrated the LLM model to post the feedback as a comment.

In [ ]:
import os
from docx import Document
from huggingface_hub import InferenceClient

# STEP 1: Set your Hugging Face token directly
os.environ["HF_TOKEN"] = "Key"  # ← Replace with yours if needed

# STEP 2: Initialize InferenceClient for LLaMA 3.1
client = InferenceClient(
    model="meta-llama/Llama-3.1-8B-Instruct",
    token=os.environ["HF_TOKEN"]
)

# STEP 3: Extract Q&A from Word
def extract_qa(filepath):
    doc = Document(filepath)
    pairs = []
    q, a = None, ""
    for para in doc.paragraphs:
        txt = para.text.strip()
        if txt.startswith("Q"):
            if q:
                pairs.append((q, a.strip()))
            q, a = txt, ""
        elif txt.startswith("A"):
            a += txt[1:].strip()
        else:
            a += "\n" + txt
    if q:
        pairs.append((q, a.strip()))
    return pairs, doc

# STEP 4: Generate feedback using hosted LLaMA
def generate_feedback_llama(q, a):
    prompt = f"""
You are a safeguarding training assessor. Provide structured feedback for the answer below:

Question: {q}
Answer: {a}

Format:
✅ Positive points
👏 Praise (if any)
⚠️ Suggestions for improvement (if any)
"""
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=256,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# STEP 5: Add feedback as comments to Word file
def add_feedback_as_comments(doc, qa_pairs):
    for i, (q, a) in enumerate(qa_pairs, 1):
        feedback = generate_feedback_llama(q, a)
        # Find the paragraph starting with Q and add comment
        for para in doc.paragraphs:
            if para.text.strip().startswith(f"Q{i}."):
                if para.runs:  # Ensure there are runs
                    first_run = para.runs[0]  # First run of the paragraph
                    last_run = para.runs[-1] if len(para.runs) > 1 else first_run  # Last run
                    # Add comment with the run range
                    comment = doc.add_comment(runs=[first_run, last_run], text=feedback, author="Ancy")
                    # Ensure comment is properly placed (no need for manual range markup with runs provided)
                break

# STEP 6: Run the full pipeline
qa_pairs, doc = extract_qa("Q1.docx")  # Adjust if filename changed
add_feedback_as_comments(doc, qa_pairs)
doc.save("Q1.docx")
print("✅ Feedback added as comments to Q1.docx")

✅ Feedback added as comments to Q1.docx
